Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (0.002*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 23.039101
Minibatch accuracy: 14.1%
Validation accuracy: 16.3%
Minibatch loss at step 500: 2.581672
Minibatch accuracy: 79.7%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 1.409321
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 0.667384
Minibatch accuracy: 86.7%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 0.608162
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.728481
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.711199
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Test accuracy: 89.1%


In [9]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables for layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Computation at hidden layer
  hidden_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
    
  # Variables for layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(hidden_out, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + (0.002*tf.nn.l2_loss(weights1)) + (0.001*tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 954.402588
Minibatch accuracy: 5.5%
Validation accuracy: 27.0%
Minibatch loss at step 500: 235.726715
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 83.837059
Minibatch accuracy: 78.9%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 30.764286
Minibatch accuracy: 90.6%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 11.584848
Minibatch accuracy: 90.6%
Validation accuracy: 86.3%
Minibatch loss at step 2500: 4.633709
Minibatch accuracy: 88.3%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 2.141381
Minibatch accuracy: 83.6%
Validation accuracy: 86.8%
Test accuracy: 93.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [49]:
batch_size = 20
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables for layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Computation at hidden layer
  hidden_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
    
  # Variables for layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(hidden_out, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (0.001*tf.nn.l2_loss(weights1)) + (0.001*tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [50]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 697.363464
Minibatch accuracy: 20.0%
Validation accuracy: 14.7%
Minibatch loss at step 500: 348.481873
Minibatch accuracy: 70.0%
Validation accuracy: 70.0%
Minibatch loss at step 1000: 279.536438
Minibatch accuracy: 70.0%
Validation accuracy: 65.7%
Minibatch loss at step 1500: 75.969032
Minibatch accuracy: 70.0%
Validation accuracy: 73.2%
Minibatch loss at step 2000: 46.006050
Minibatch accuracy: 70.0%
Validation accuracy: 76.6%
Minibatch loss at step 2500: 27.914904
Minibatch accuracy: 80.0%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 17.164928
Minibatch accuracy: 75.0%
Validation accuracy: 81.5%
Test accuracy: 88.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [247]:
batch_size = 128
hidden_nodes = 1024

b1 = 0.001
b2 = 0.001
kb = 0.95
alpha = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables for layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Computation at hidden layer
  hidden_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
  hidden_out = tf.nn.dropout(hidden_out, keep_prob=kb)
    
  # Variables for layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(hidden_out, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (b1*tf.nn.l2_loss(weights1)) + (b2*tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [248]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 647.215881
Minibatch accuracy: 10.2%
Validation accuracy: 36.7%
Minibatch loss at step 500: 194.876877
Minibatch accuracy: 85.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 115.310532
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 68.477318
Minibatch accuracy: 86.7%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 41.188259
Minibatch accuracy: 90.6%
Validation accuracy: 84.4%
Minibatch loss at step 2500: 25.122084
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 3000: 15.478789
Minibatch accuracy: 86.7%
Validation accuracy: 86.4%
Minibatch loss at step 3500: 9.557264
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Minibatch loss at step 4000: 5.964285
Minibatch accuracy: 90.6%
Validation accuracy: 87.7%
Minibatch loss at step 4500: 3.792960
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Test accuracy: 93.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [249]:
batch_size = 128
hidden_nodes = 1024

b1 = 0.001
b2 = 0.001
kb = 0.75

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.8
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           500, 0.96, staircase=True)
  
  # Variables for layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Computation at hidden layer
  hidden_out = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
  hidden_out = tf.nn.dropout(hidden_out, keep_prob=kb)
    
  # Variables for layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(hidden_out, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (b1*tf.nn.l2_loss(weights1)) + (b2*tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [250]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 700.512451
Minibatch accuracy: 16.4%
Validation accuracy: 26.4%
Minibatch loss at step 500: 150.533356
Minibatch accuracy: 71.9%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 65.093880
Minibatch accuracy: 76.6%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 30.585009
Minibatch accuracy: 89.1%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 15.270488
Minibatch accuracy: 89.8%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 8.051443
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 4.483682
Minibatch accuracy: 85.9%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 2.704555
Minibatch accuracy: 82.8%
Validation accuracy: 86.5%
Minibatch loss at step 4000: 1.659827
Minibatch accuracy: 90.6%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 1.151197
Minibatch accuracy: 88.3%
Validation accuracy: 86.5%
Test accuracy: 93.3%


In [97]:
train_dataset.shape

(200000, 784)

In [27]:
batch_size = 2000
hidden_nodes1 = 2000
hidden_nodes2 = 500
hidden_nodes3 = 50

b1 = 0.001
b2 = 0.001
b3 = 0.001
b4 = 0.001
kb1 = 0.5
kb2 = 0.5
kb3 = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           500, 0.9, staircase=True)
  
  # Variables for layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1], stddev=0.1))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
  
  # Computation at hidden layer 1
  hidden_out1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
  hidden_out1 = tf.nn.dropout(hidden_out1, keep_prob=kb1)
    
  # Variables for layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev=0.1))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))

  # Computation at hidden layer 2
  hidden_out2 = tf.nn.relu(tf.matmul(hidden_out1, weights2) + biases2)  
  hidden_out2 = tf.nn.dropout(hidden_out2, keep_prob=kb2)

  # Variables for layer 3
  weights3 = tf.Variable(
  tf.truncated_normal([hidden_nodes2, hidden_nodes3], stddev=0.1))
  biases3 = tf.Variable(tf.zeros([hidden_nodes3]))

  # Computation at hidden layer 3
  hidden_out3 = tf.nn.relu(tf.matmul(hidden_out2, weights3) + biases3)  
  hidden_out3 = tf.nn.dropout(hidden_out3, keep_prob=kb3)

  # Variables for layer 4
  weights4 = tf.Variable(
  tf.truncated_normal([hidden_nodes3, num_labels], stddev=0.1))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(hidden_out3, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (b1*tf.nn.l2_loss(weights1)) + (b2*tf.nn.l2_loss(weights2)) + (b3*tf.nn.l2_loss(weights3)) + (b4*tf.nn.l2_loss(weights4))
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  vp1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  vp2 = tf.nn.relu(tf.matmul(vp1, weights2) + biases2)
  vp3 = tf.nn.relu(tf.matmul(vp2, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(vp3, weights4) + biases4)
  
  tp1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  tp2 = tf.nn.relu(tf.matmul(tp1, weights2) + biases2)
  tp3 = tf.nn.relu(tf.matmul(tp2, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(tp3, weights4) + biases4)

In [28]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.584894
Minibatch accuracy: 9.8%
Validation accuracy: 10.2%
Minibatch loss at step 500: 6.944527
Minibatch accuracy: 75.2%
Validation accuracy: 82.6%
Minibatch loss at step 1000: 4.323964
Minibatch accuracy: 82.2%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 2.858538
Minibatch accuracy: 83.5%
Validation accuracy: 86.5%
Minibatch loss at step 2000: 1.865432
Minibatch accuracy: 86.2%
Validation accuracy: 88.0%
Minibatch loss at step 2500: 1.330404
Minibatch accuracy: 87.1%
Validation accuracy: 88.6%
Minibatch loss at step 3000: 0.966072
Minibatch accuracy: 89.0%
Validation accuracy: 89.1%
Minibatch loss at step 3500: 0.780235
Minibatch accuracy: 88.6%
Validation accuracy: 89.5%
Minibatch loss at step 4000: 0.678040
Minibatch accuracy: 89.2%
Validation accuracy: 89.9%
Minibatch loss at step 4500: 0.569087
Minibatch accuracy: 90.5%
Validation accuracy: 89.7%
Minibatch loss at step 5000: 0.548511
Minibatch accuracy: 89.8%
Validation accurac

In [223]:
batch_size = 150
hidden_nodes1 = 800
hidden_nodes2 = 300

b1 = 0.008
b2 = 0.005
b3 = 0.005
kb1 = 0.75
kb2 = 0.75

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.6
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           500, 0.9, staircase=True)
  
  # Variables for layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
  
  # Computation at hidden layer 1
  hidden_out1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)  
  hidden_out1 = tf.nn.dropout(hidden_out1, keep_prob=kb1)
    
  # Variables for layer 2
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2]))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))

  # Computation at hidden layer 2
  hidden_out2 = tf.nn.relu(tf.matmul(hidden_out1, weights2) + biases2)  
  hidden_out2 = tf.nn.dropout(hidden_out2, keep_prob=kb2)

  # Variables for layer 3
  weights3 = tf.Variable(
  tf.truncated_normal([hidden_nodes2, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(hidden_out2, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + (b1*tf.nn.l2_loss(weights1)) + (b2*tf.nn.l2_loss(weights2)) + (b3*tf.nn.l2_loss(weights3))
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  vp1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  vp2 = tf.nn.relu(tf.matmul(vp1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(vp2, weights3) + biases3)
  
  tp1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  tp2 = tf.nn.relu(tf.matmul(tp1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tp2, weights3) + biases3)

In [224]:
num_steps = 6000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6323.911621
Minibatch accuracy: 12.0%
Validation accuracy: 24.0%
Minibatch loss at step 500: 1451.605347
Minibatch accuracy: 11.3%
Validation accuracy: 10.5%
Minibatch loss at step 1000: 738.226257
Minibatch accuracy: 12.7%
Validation accuracy: 16.6%
Minibatch loss at step 1500: 384.223755
Minibatch accuracy: 23.3%
Validation accuracy: 25.2%
Minibatch loss at step 2000: 204.643021
Minibatch accuracy: 34.7%
Validation accuracy: 40.6%
Minibatch loss at step 2500: 111.533569
Minibatch accuracy: 50.0%
Validation accuracy: 61.3%
Minibatch loss at step 3000: 61.981441
Minibatch accuracy: 65.3%
Validation accuracy: 76.7%
Minibatch loss at step 3500: 35.270931
Minibatch accuracy: 70.0%
Validation accuracy: 81.1%
Minibatch loss at step 4000: 20.438429
Minibatch accuracy: 78.0%
Validation accuracy: 83.1%
Minibatch loss at step 4500: 12.186731
Minibatch accuracy: 79.3%
Validation accuracy: 84.2%
Minibatch loss at step 5000: 7.395693
Minibatch accuracy: 82.0%
